# Prepare training and test data for specific tools

In [ ]:
import json

import pandas as pd
import numpy as np
import scanpy as sc
import anndata as adata

from tqdm import tqdm
from pathlib import Path
from sklearn import preprocessing as pp
from sklearn.preprocessing import LabelEncoder

%load_ext blackcellmagic

In [ ]:
# Working directory
prefix = "???/deconvolution_benchmarking/04_tcga_bulk_validation"

In [ ]:
# Training patient IDs
train_p_ids = [
    "CID3586",
    "CID3941",
    "CID3963",
    "CID44041",
    "CID4530N",
    "CID3838",
    "CID3946",
    "CID4040",
    "CID4461",
    "CID44991",
    "CID45171",
    "CID4535",
    "CID3948",
    "CID4398",
    "CID4463",
    "CID4495",
    "CID4513",
    "CID4465",
]
# Training patient IDs
test_p_ids = [
    "CID4067",
    "CID4290A",
    "CID4471",
    "CID3921",
    "CID4066",
    "CID4523",
    "CID44971",
    "CID4515",
]
# Cell types
c_types = [
    "B-cells",
    "CAFs",
    "Cancer Epithelial",
    "Endothelial",
    "Myeloid",
    "Normal Epithelial",
    "PVL",
    "Plasmablasts",
    "T-cells",
]

## 1. Load single-cell and bulk counts

#### Load single-cell counts

In [ ]:
# Raw counts
sc_adata = adata.read_h5ad(
    Path(prefix).joinpath("data/filtered/intersect/scRNA_ref_raw.h5ad")
)
sc_df = sc_adata.to_df()

# Counts-per-10,000 counts
normalized_sc_adata = sc.read_h5ad(
    Path(prefix).joinpath("data/filtered/intersect/scRNA_ref_normalized.h5ad")
)
normalized_sc_df = normalized_sc_adata.to_df()

In [ ]:
# We pre-filtered metadata file to remove cells type with count < 10 per patient
# Load pre-filtered metadata and filter sc_df and normalized_sc_df based on it
# First load up all metadata created by Seurat
meta_df = pd.read_csv(
    Path(prefix).joinpath("data/deconv/Whole_miniatlas_meta_9_10.csv"),
    index_col=0,
    sep="\t",
)
train_meta_df = meta_df[meta_df["Patient"].isin(train_p_ids)]

# Filter counts and metadata
train_sc_df = sc_df.loc[train_meta_df.index, :].copy()
train_normalized_sc_df = normalized_sc_df.loc[train_meta_df.index, :].copy()

#### Load TCGA bulk counts

In [ ]:
# Raw counts
tcga_raw_counts_df = pd.read_csv(
    Path(project_prefix).joinpath("data/filtered/intersect/tcga_raw_counts.csv"),
    index_col=0,
    sep="\t",
)

# TPM counts
tcga_tpm_counts_df = pd.read_csv(
    Path(project_prefix).joinpath("data/filtered/intersect/tcga_tpm_counts.csv"),
    index_col=0,
    sep="\t",
)

## 2. Prepare single-cell reference for each method

#### CIBERSORTx

In [ ]:
# Convert major cell type mapping into dict
major_ctype_mapping = {
    row[0]: row[1] for i, row in train_meta_df[["cell_labels"]].reset_index().iterrows()
}
# Replace cell ids with cell types
cbx_sc_df = train_normalized_sc_df.T.rename(columns=major_ctype_mapping)

In [ ]:
# Save output beautifully
cbx_sc_df.to_csv(
    Path(prefix).joinpath(f"data/deconv/cbx/scRNA_ref.tsv"),
    sep="\t",
    chunksize=5000,
)

#### Scaden
We use simulated mixtures generated for the matching pseudobulk project

In [ ]:
# Prefix to scaden data folder in the matching pseudobulk project
pseudobulk_prefix = (
    "???/01_purity_levels_experiment/include_normal_epithelial/data/scaden"
)

In [ ]:
# Load train AnnData object
scaden_train_adata = adata.read_h5ad(
    Path(pseudobulk_prefix).joinpath("train_counts.h5ad")
)
scaden_train_df = scaden_train_adata.to_df()

# Load hugo-ensembl mapping
hugo_ensembl_mapping_df = pd.read_csv(
    Path(project_prefix).joinpath("data/raw/hugo_ensembl_maps.tsv"),
    header=None,
    sep="\t",
)
hugo_ensembl_mapping_df.columns = ["gene_symbol", "ensembl_id"]
hugo_ensembl_mapping_d = {
    i[1]["gene_symbol"]: i[1]["ensembl_id"] for i in hugo_ensembl_mapping_df.iterrows()
}

# Replace gene_symbol with ensembl_id in scaden_train_df
scaden_train_df.rename(columns=hugo_ensembl_mapping_d, inplace=True)

# Only keep ensembl genes that exist in filtered single-cell dataframe
filtered_scaden_train_df = scaden_train_df[sc_df.columns]

In [ ]:
# Create and save filtered AnnData object
filtered_train_adata = adata.AnnData(
    X=filtered_scaden_train_df.values,
    obs=scaden_train_adata.obs.copy(),
    var=filtered_scaden_train_df.columns.to_frame().rename(
        columns={"index": "ensembl_id"}
    ),
    dtype="float64",
)

# Scaden requires cell fractions DataFrame to have a column call "ds"
# This column is supposed to store info on what dataset each row comes from
# And the during training we can delect which dataset gets used for training, which is quite handy
# However, in this case, there is only 1 dataset
# Make all row ds="Swarbrick_GSE176078"
filtered_train_adata.obs["ds"] = "Swarbrick_GSE176078"

# add cell types and signature genes
filtered_train_adata.uns["cell_types"] = c_types

# Rename index and columns properly
filtered_train_adata.obs.index.name = "mixture_id"
filtered_train_adata.var.index.name = None

# Save AnnData object
filtered_train_adata.write_h5ad(f"{prefix}/data/deconv/scaden/train_counts.h5ad")

#### CPM

For CPM, we need to prepare 3 files (in addition to bulk counts):
- single-cell reference:    rows as genes, columns as cells
- cell labels:              one single column with cell labels
- UMAP/tSNE:                first column is cell labels, next 2 columns are UMAP/tSNE coordinates

#### UMAP coordinates
CPM requires the cell state space to be dense and smooth, as well as able to reflect the phenotype of cells and capture the essence of gene-regulation variation among the reference single cells. One way to generate such cell-state space is via the use of dimensionality reduction techniques such as tSNE or UMAP.

Wu et al has done exceptional work in integrating single cells from 26 patients and across 3 different molecular subtypes in their dimensional reduction analysis. This abundance of cells from multiple subjects and subtypes resulted in a very dense and smooth distributions of cells in 2-dimensional space. We took advantages of the UMAP coordinates from this analysis, filtered out cells from test patients, i.e. those used for simulated text bulk mixtures. UMAP coordinates from the retained cells (i.e. cells from training patients) were used as cell-state space for UMAP.

Moreover, we attempted at re-running Wu et al's dimensional reduction pipeline on only cells from training patients and produced very similar UMAP distributions compared to the original UMAP coordinates. On the other hand, CPM's requirements for the cell-state space suggests that the quality of this space can benefit from larger and more diverse cells. For these two reasons, we chose to employ the original UMAP coordinates provided by Wu et al. (after cells from test patients are filtered out).

In [ ]:
# First load up all manifold coordinates created by Wu etl
umap_df = pd.read_csv(
    Path(prefix).joinpath("data/Whole_miniatlas_umap.coords.tsv"), index_col=0, sep="\t"
)

# Drop second row which contains datatype
umap_df.drop(["TYPE"], axis=0, inplace=True)
umap_df = umap_df.astype(float)

#### Random sample 1,330 cells for each cell type
- We have 59,680 single cells in the training data
- With CPM's settings, and with the way we parallelize the execution into 19 partitions, each partition will take ~30hours to finish
- There are only 3 machines in HPC that can help us achieve this performance
- So **30h * 19 = 570h** in totals. Split across 3 machines, ie. **570 / 3 = 190h**, or 8 days to finish one run (given we can have these 3 nodes unteruptedly for 8 days)
- This is why current results are being generated using only 11,969 cells (i.e. 1,329 cells per type). The full run is still being processed while we push ahead with drafting the paper

In [ ]:
# First random sample 1,330 cells from each type
l = []

for c_type in tqdm(c_types):
    subset_train_meta_df = train_meta_df[train_meta_df["cell_labels"] == c_type]
    l.append(subset_train_meta_df.sample(n=1329, random_state=41))

sampled_train_meta_df = pd.concat(l, axis=0)
sampled_cell_ids = sampled_train_meta_df.index.tolist()

In [ ]:
# Filter single cell labels
sampled_sc_labels_df = sampled_train_meta_df.sort_index()["cell_labels"].to_frame()

# Filter single cell labels
sampled_umap_df = umap_df[umap_df.index.isin(sampled_cell_ids)].sort_index()

In [ ]:
# Use these cell ids to filter counts data
cpm_sc_df = (
    train_normalized_sc_df[train_normalized_sc_df.index.isin(sampled_cell_ids)]
    .sort_index()
    .T
)

# Add cell types to single cell reference matrix columns
cpm_sc_df.columns = (
    meta_df[meta_df.index.isin(cpm_sc_df.T.index)]["cell_labels"].sort_index().values
    + "_"
    + cpm_sc_df.T.index.values
)
cpm_sc_df.index.name = "gene_symbol"

In [ ]:
# Use this save function if we are using original cell-state from Wu et al and only 1,330 cells per type
experiment = "expr_2_original_cellstate_1330_per_ctype"

sampled_sc_labels_df.to_csv(
    Path(prefix).joinpath(f"data/cpm/{experiment}/single_cell_label.csv"), sep="\t"
)

sampled_umap_df.to_csv(
    Path(prefix).joinpath(f"data/cpm/{experiment}/cell_state.csv"), sep=","
)

cpm_sc_df.to_csv(
    Path(prefix).joinpath(f"data/cpm/{experiment}/scRNA_ref_1330_per_ctype.txt"),
    sep="\t",
    chunksize=1000,
)

#### bisque
bisque expect a .h5ad file holding non-logs single-cell gene counts in the bique/ folder <br>
This file would have been previously generated for CPM

In [ ]:
# Extract patieint id, cell labels and cell ids into a phenotype DataFrame
pheno_df = train_meta_df[["Patient", "cell_labels"]].reset_index()
pheno_df.columns = ["cell_ids", "patient_ids", "cell_labels"]

pheno_df.to_csv(Path(prefix).joinpath("data/deconv/bisque/phenotypes.csv"), sep="\t")

#### We're using scaled non-logged counts

In [ ]:
# Re-arrange single-cell DataFrame to match the same order of cell ids as phenotype DataFrame
bisque_sc_df = train_normalized_sc_df.T[pheno_df["cell_ids"].values].copy()

# Normalize data
mms = pp.MinMaxScaler(feature_range=(0, 1), copy=True)
bisque_scaled_sc_arr = mms.fit_transform(bisque_sc_df.T).T
bisque_scaled_sc_df = pd.DataFrame(
    bisque_scaled_sc_arr, index=bisque_sc_df.index, columns=bisque_sc_df.columns
)

# Save scaled linear counts
bisque_scaled_sc_df.to_csv(
    Path(prefix).joinpath("data/bisque/scaled_scRNA_ref.csv"), sep="\t", chunksize=5000
)

#### DWLS
DWLS only expects single cell labels accompanying the single-cell data

In [ ]:
# Extract cell labels into a DataFrame
labels_df = train_meta_df[["cell_labels"]].sort_index()

# Apparently R/3.5.0 doesn't understand how to parse the character "-"
# meaning "T-cells" will be read as a vector of "T" and "cells"
# Also R/3.5.0 can't parse " "
# Replace all cell types with these characters by "_"
labels_df["cell_labels"].replace(
    {
        "T-cells": "T_cells",
        "B-cells": "B_cells",
        "Normal Epithelial": "Normal_Epithelial",
        "Cancer Epithelial": "Cancer_Epithelial",
    },
    inplace=True,
)

labels_df.to_csv(
    Path(prefix).joinpath("data/deconv/dwls/single_cell_labels.csv"), sep="\t"
)

In [ ]:
# Re-arrange single-cell DataFrame to match the same order of cell ids as phenotype DataFrame
dwls_sc_df = train_normalized_sc_df.T[labels_df.index].copy()

dwls_sc_df.to_csv(
    Path(prefix).joinpath("data/deconv/dwls/scRNA_ref.csv"), sep="\t", chunksize=5000
)

#### EPIC
EPIC relies on the signature matrix and marker genes generated by CIBERSORTx to run <br>
This processing script assumes that these 2 files have already been put in the data/epic folders
- Signature matrix (containing all genes): cbx_sig_matrix.txt
- Marker genes (a subset of signature matrix): cbx_sig_matrix.txt

In [ ]:
# Load signature matrix and marker genes profiles
cbx_sig_matrix_df = pd.read_csv(
    Path(prefix).joinpath("data/deconv/epic/cbx_sig_matrix/cbx_sig_matrix.txt"),
    index_col=0,
    sep="\t",
)

# EPIC assumes the "unknown" cells in a tumour is cancer cells
# Therefore we need to drop Cancer Epithelial from the signature matrix
cbx_sig_matrix_df.drop(["Cancer Epithelial"], axis=1, inplace=True)

# Save signature matrix beautifully
cbx_sig_matrix_df.to_csv(
    Path(prefix).joinpath("data/deconv/epic/cbx_sig_matrix/reference_profiles.csv"),
    sep="\t",
)

# Extract marker genes from marker gene profiles and save into a .csv
marker_gene_labels_df = (
    cbx_sig_matrix_df.index.to_frame()
    .rename(columns={"GeneSymbol": "gene_symbol"})
    .reset_index()
    .drop(["GeneSymbol"], axis=1)
)

marker_gene_labels_df.to_csv(
    Path(prefix).joinpath("data/deconv/epic/cbx_sig_matrix/marker_gene_symbols.csv"),
    sep="\t",
)

#### hspe

In [ ]:
# Apply log1p (i.e. add 1 and apply log2)
# hspe only mentions log2 without + 1. This will lead to undefined output, as log2(0) = infinity. We therefore added 1 to gene expressions to avoid this
# 0 gene expression values will stil return 0 after log1p transformation
hspe_sc_df = np.log2(train_normalized_sc_df + 1)

# Rename index
hspe_sc_df.columns.name = "gene_symbol"

In [ ]:
# Apply log1p one TCGA TPM counts
hspe_test_counts_df = np.log2(tcga_tpm_counts_df.T + 1)
hspe_test_counts_df.columns.name = "gene_symbol"

##### Save train & test counts

In [ ]:
# Before saving train and test counts , do a sanity check to make sure train and test DataFrames have the same genes in the same order
assert np.array_equal(
    hspe_test_counts_df.columns.to_numpy(), hspe_test_counts_df.columns.to_numpy()
)

In [ ]:
# Save single-cell datta
hspe_sc_df.to_csv(Path(prefix).joinpath("data/deconv/hspe/scRNA_ref.csv"), sep="\t")

In [ ]:
# Split TCGA data into 10 shards
# This allows us to paralellize the run into 190-fold
for shard in tqdm(list(range(0, 20, 1))):
    shard_hspe_test_counts_df = np.array_split(hspe_test_counts_df, 20)[shard]

    shard_hspe_test_counts_df.to_csv(
        Path(prefix).joinpath(f"data/deconv/hspe/logged_test_counts_{shard}.txt"),
        sep="\t",
    )

##### Extract pure samples
Both dtangle and hspe require a pure_samples variable. This is a list variable, in which each item corresponds to one cell type and indexes of all cells of the same type in the single-cell reference DataFrame <br>

We need to retrieve cell type of the single-cell reference data and save this information into a .json file

In [ ]:
# Reset index of log_train_sc_df() so we have order of cell ids as the indexes
reset_hspe_sc_df = hspe_sc_df.reset_index().rename(columns={"NAME": "cell_ids"})

# Iterate over cell types and extract cell indexes from single-cell reference
pure_samples_d = {}

for c_type in tqdm(train_meta_df["cell_labels"].unique()):
    c_ids = (train_meta_df[train_meta_df["cell_labels"] == c_type]).index.tolist()
    c_indexes = reset_hspe_sc_df[reset_hspe_sc_df["cell_ids"].isin(c_ids)].index

    # Python starts indexes from 0 and R starts from 1
    # Add 1 to index and add to pure_samples_d
    pure_samples_d[c_type] = (c_indexes + 1).tolist()

# Remap keys containing spaces and hyphens
pure_samples_d["T_cells"] = pure_samples_d.pop("T-cells")
pure_samples_d["B_cells"] = pure_samples_d.pop("B-cells")
pure_samples_d["Normal_Epithelial"] = pure_samples_d.pop("Normal Epithelial")
pure_samples_d["Cancer_Epithelial"] = pure_samples_d.pop("Cancer Epithelial")

In [ ]:
# Save pure_samples_d into a json file
json.dump(
    pure_samples_d,
    open(Path(prefix).joinpath(f"data/deconv/hspe/pure_samples.json"), "w"),
    indent=4,
)

### 8. MuSiC
MuSiC requires single-cell and bulk expressions in ExpressionSet objects <br>
The single-cell ExpressionSet also needs to a phenoType item containing
- **sampleID**        index of patient
- **SubjectName**      patient id
- **cellTypeID**       index of cell type
- **cellType**         cell annotation labels

In [ ]:
# Rename index
music_sc_df = train_normalized_sc_df.T.copy()
music_sc_df.index.name = "gene_symbol"

In [ ]:
# Extract "Patient" + "celltype_major columns" and rename columns to match MuSiC requirements
pheno_df = train_meta_df[["Patient", "cell_labels"]].rename(
    columns={"Patient": "SubjectName", "cell_labels": "cellType"}
)

pheno_df.index.name = None

In [ ]:
# Encode cell labels into number to use as cellTypeID
l_encoder = LabelEncoder()
l_encoder.fit(c_types)
pheno_df["cellTypeID"] = l_encoder.transform(pheno_df["cellType"]) + 1

# Encode patient ids into number to use as sampleID
l_encoder = LabelEncoder()
l_encoder.fit(pheno_df["SubjectName"].unique())
pheno_df["sampleID"] = l_encoder.transform(pheno_df["SubjectName"]) + 1

In [ ]:
# Save pheno DataFrame
pheno_df.to_csv(Path(prefix).joinpath("data/deconv/music/pheno.csv"), sep="\t")

# Save train counts
music_sc_df.to_csv(
    Path(prefix).joinpath("data/deconv/music/scRNA_ref.csv"), sep="\t", chunksize=5000
)

### 9. BayesPrism

In [ ]:
# Copy train single-cell to bprism_sc_df
bprism_sc_df = train_normalized_sc_df.copy()

In [ ]:
# Extract cell labels into a DataFrame
labels_df = train_meta_df[["cell_labels", "celltype_minor"]]

# Apparently R/4.2.0 doesn't understand how to parse the character "-"
# meaning "T-cells" will be read as a vector of "T" and "cells"
# Also R/4.2.0 can't parse " "
# Replace all cell types with these characters by "_"
labels_df["cell_labels"].replace(
    {
        "T-cells": "T_cells",
        "B-cells": "B_cells",
        "Cancer Epithelial": "Cancer_Epithelial",
        "Normal Epithelial": "Normal_Epithelial",
    },
    inplace=True,
)
labels_df["celltype_minor"].replace(
    {
        "Endothelial ACKR1": "Endothelial_ACKR1",
        "Endothelial RGS5": "Endothelial_RGS5",
        "Endothelial CXCL12": "Endothelial_CXCL12",
        "CAFs MSC iCAF-like": "CAFs_MSC_iCAF-like",
        "CAFs myCAF-like": "CAFs_myCAF_like",
        "PVL Differentiated": "PVL_Differentiated",
        "PVL Immature": "PVL_Immature",
        "Endothelial Lymphatic LYVE1": "Endothelial_Lymphatic_LYVE1",
        "B cells Memory": "B_cells_Memory",
        "B cells Naive": "B_cells_Naive",
        "T cells CD8+": "T_cells_CD8",
        "T cells CD4+": "T_cells_CD4",
        "NK cells": "NK_cells",
        "Cycling T-cells": "Cycling_T_cells",
        "NKT cells": "NKT_cells",
        "Luminal Progenitors": "Luminal_Progenitors",
        "Mature Luminal": "Mature_Luminal",
        "Cycling PVL": "Cycling_PVL",
        "Cancer LumB SC": "Cancer_LumB_SC",
        "Cancer Cycling": "Cancer_Cycling",
        "Cancer LumA SC": "Cancer_LumA_SC",
        "Cancer Basal SC": "Cancer_Basal_SC",
        "Cancer Her2 SC": "Cancer_Her2_SC",
    },
    inplace=True,
)

# Rename column
labels_df.rename(
    columns={"cell_labels": "cell_type_labels", "celltype_minor": "cell_state_labels"},
    inplace=True,
)

In [ ]:
# Save single-cell counts and labels
labels_df.to_csv(
    Path(prefix).joinpath("data/deconv/bprism/single_cell_labels.csv"), sep="\t"
)
bprism_v2_sc_df.to_csv(
    Path(prefix).joinpath("data/deconv/bprism/scRNA_ref.csv"), sep="\t", chunksize=5000
)